In [1]:
class Node:
    def __init__(self, data, depth=0, lchild=None, rchild=None):
        self.data = data
        self.depth = depth
        self.lchild = lchild
        self.rchild = rchild


class KdTree:
    def __init__(self):
        self.KdTree = None
        self.n = 0
        self.nearest = None

    def create(self, dataSet, depth=0):
        if len(dataSet) > 0:
            m, n = np.shape(dataSet)
            self.n = n - 1
            axis = depth % self.n
            mid = int(m / 2)
            dataSetcopy = sorted(dataSet, key=lambda x: x[axis])
            node = Node(dataSetcopy[mid], depth)
            if depth == 0:
                self.KdTree = node
            node.lchild = self.create(dataSetcopy[:mid], depth+1)
            node.rchild = self.create(dataSetcopy[mid+1:], depth+1)
            return node
        return None

    def preOrder(self, node):
        if node is not None:
            print(node.depth, node.data)
            self.preOrder(node.lchild)
            self.preOrder(node.rchild)

    def search(self, x, count=1):
        nearest = []
        for i in range(count):
            nearest.append([-1, None])
        self.nearest = np.array(nearest)

        def recurve(node):
            if node is not None:
                axis = node.depth % self.n
                daxis = x[axis] - node.data[axis]
                if daxis < 0:
                    recurve(node.lchild)
                else:
                    recurve(node.rchild)

                dist = sqrt(sum((p1 - p2) ** 2 for p1, p2 in zip(x, node.data)))
                for i, d in enumerate(self.nearest):
                    if d[0] < 0 or dist < d[0]:
                        self.nearest = np.insert(self.nearest, i, [dist, node], axis=0)
                        self.nearest = self.nearest[:-1]
                        break

                n = list(self.nearest[:, 0]).count(-1)
                if self.nearest[-n-1, 0] > abs(daxis):
                    if daxis < 0:
                        recurve(node.rchild)
                    else:
                        recurve(node.lchild)

        recurve(self.KdTree)

        knn = self.nearest[:, 1]
        return self.nearest

In [2]:
import numpy as np
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits.axisartist.axislines as axislines
import matplotlib as mpl
from sklearn import preprocessing
min_max_scaler=preprocessing.MinMaxScaler()
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
from sklearn.model_selection import KFold
kf=KFold(n_splits=10)

def DDF(df):    
    data2=np.array(df.drop('label',axis=1))
    dmax=[]
    for m in range(0,len(data2)-1):
        for n in range(m+1,len(data2)):
            dmax.append(sqrt(sum((data2[m] - data2[n]) ** 2 )))
    a = np.percentile(dmax,10) #密度阈值(%10分位数)
    k=7
    x1 = np.array([[1,1],[1,-1]])
    x2 = np.array([1,np.floor(k / 3)/ k])
    (Ps_,Pd_)=np.linalg.solve(x1,x2)
    H_ = -Ps_* np.log2(Ps_)-Pd_*np.log2(Pd_) #标签混合程度阈值
    Fh,Re=[],[]
    data=np.array(df)
    for rate in [0.05,0.1,0.2,0.3,0.4]:
        fh_temp,re_temp=[],[]
        for train_index, test_index in kf.split(data):
            train=data[train_index]
            test=data[test_index]
            train=pd.DataFrame(train)
            train.columns=df.columns
            Dn= []
            noise_set=pd.DataFrame(columns=train.columns)
            label_set=list(set(train.label))
            for r in label_set:    
                noise_set=pd.concat([noise_set,train[train.label==r].sample(frac=rate,replace=False)]) #选取噪声比例 
            train_cut=train[~train.index.isin(noise_set.index)]
            no=[]
            for j in range(0,len(noise_set)):
                no.append(np.random.choice(label_set))
            noise_set['label2']=no
            noise_set.reset_index(drop=True,inplace=True)
            for l in range(0,len(noise_set)):  #随机替换标签
                label_set=list(set(train.label))
                if (noise_set.loc[l,'label']==noise_set.loc[l,'label2']):
                    label_set.remove(noise_set.loc[l,'label'])
                    noise_set.loc[l,'label2']=np.random.choice(label_set) #从剩余列表中随机选择
            noise=noise_set.drop(['label'],axis=1)
            noise.rename(columns={'label2':'label'}, inplace = True)
            train= pd.concat([train_cut,noise],axis=0,ignore_index=True) #含噪声数据集
            train_set=np.array(train)
            kdt = KdTree()
            kdt.create(train_set)  
            for x in train_set:
                near= kdt.search(x[:-1], k+1)  # 设置临近点的个数
                density = 0
                t,hon,hen,dist_hon,dist_hen,DoD,DRL= 0,0,0,0,0,0,0
                for i in range(1,k+1):
                    if x[-1] == near[i][1].data[-1]:
                        t += 1
                        hon += 1
                        dist_hon += near[i][0]
                    else:
                        hen += 1
                        dist_hen += near[i][0]

                    density += near[i][0]  #密度   
                DoD = abs(dist_hen - dist_hon) #相异性差值(要加绝对值)
                DRL = (hen - hon)  #标签异同差
                Ps = t / k
                Pd = 1 - Ps
                if (Ps==1)|(Ps==0):
                    H = 0
                else:
                    H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)
              #  print('densit:',density)
                if (density <= a): #高密度区域
                   # print('高密度区')
                    if DRL > 0:
                        Dn.append(list(x))
                       # print('高密度区噪声')
                else:
                    if (H < H_): #低密度单一标签区域
                        if DRL > 0:
                            Dn.append(list(x))
                           # print('低密度单一标签噪声'
                    else: #低密度混合标签区  
                      #  print('DoD',DoD)
                        DoD_ = density / k
                       # print('DoD阈值:',DoD_)
                        if DoD >= DoD_:
                            hon_sa,hen_sa,DRL_sa=0,0,0
                            near2=[]
                            for i in range(1,k+1):
                                if near[i][0] <= a: #该点到待测点距离小于a
                                    near2.append(near[i])
                            for n2 in near2:
                                if (x[-1]==n2[1].data[-1]):
                                    hon_sa +=1
                                else:
                                    hen_sa +=1
                            DRL_sa = hen_sa - hon_sa
                            if DRL_sa > 0:        
                                Dn.append(list(x))
                               # print('混合标签区噪声')
                            elif DRL_sa ==0 and len(near2)<k:
                                if x[-1] != near[len(near2)+1][1].data[-1]:
                                    Dn.append(list(x))  

            noise_list=noise.values.tolist()
            TP = 0
            for c in Dn:
                if c in noise_list:
                    TP += 1
            FP = len(Dn) - TP
            FN = len(noise_list) - TP
            precision= TP / (TP + FP)
            recall= TP / (TP + FN)
            fh= 1.25* (precision * recall) / (0.25*precision + recall) #F0.5值
            re = len(Dn) / len(train_set)#移除率
            fh_temp.append(fh)
            re_temp.append(re)
        Fh.append(np.mean(fh_temp))
        Re.append(np.mean(re_temp))
    return Fh,Re

In [3]:
def RDS(df):    
    k=7
    x1 = np.array([[1,1],[1,-1]])
    x2 = np.array([1,np.floor(k / 3)/ k])
    (Ps_,Pd_)=np.linalg.solve(x1,x2)
    H_ = -Ps_* np.log2(Ps_)-Pd_*np.log2(Pd_) #标签混合程度阈值
    data2=np.array(df.drop('label',axis=1))
    dmax=[]
    for m in range(0,len(data2)-1):
        for n in range(m+1,len(data2)):
            dmax.append(sqrt(sum((data2[m] - data2[n]) ** 2 )))
    a = np.percentile(dmax,10) #密度阈值(%10分位数)
    Fh,Re=[],[]
    data=np.array(df)
    for rate in [0.05,0.1,0.2,0.3,0.4]:
        fh_temp,re_temp=[],[]
        for train_index, test_index in kf.split(data):
            train=data[train_index]
            test=data[test_index]
            train=pd.DataFrame(train)
            train.columns=df.columns
            Dn,Dn_filted= [],[]
            noise_set=pd.DataFrame(columns=train.columns)
            label_set=list(set(train.label))
            for r in label_set:    
                noise_set=pd.concat([noise_set,train[train.label==r].sample(frac=rate, replace=False)]) #选取噪声比例 
            train_cut=train[~train.index.isin(noise_set.index)]
            no=[]
            for j in range(0,len(noise_set)):
                no.append(np.random.choice(label_set))
            noise_set['label2']=no
            noise_set.reset_index(drop=True,inplace=True)
            for l in range(0,len(noise_set)):  #随机替换标签
                label_set=list(set(train.label))
                if (noise_set.loc[l,'label']==noise_set.loc[l,'label2']):
                    label_set.remove(noise_set.loc[l,'label'])
                    noise_set.loc[l,'label2']=np.random.choice(label_set) #从剩余列表中随机选择
            noise=noise_set.drop(['label'],axis=1)
            noise.rename(columns={'label2':'label'}, inplace = True)
            train= pd.concat([train_cut,noise],axis=0,ignore_index=True) #含噪声数据集
            train_set=np.array(train)
            kdt = KdTree()
            kdt.create(train_set)  
            for x in train_set:
                near = kdt.search(x[:-1], k+1)  # 设置临近点的个数
                density = 0
                t,hon,hen,dist_hon,dist_hen,DoD,DRL= 0,0,0,0,0,0,0
                for i in range(1,k+1):
                    if x[-1] == near[i][1].data[-1]:
                        t += 1
                        hon += 1
                        dist_hon += near[i][0]
                    else:
                        hen += 1
                        dist_hen += near[i][0]
                    density += near[i][0]  #密度   

                DoD = abs(dist_hen - dist_hon) #相异性差值(要加绝对值)
                DRL = (hen - hon)  #标签异同差   
                Ps = t / k
                Pd = 1 - Ps
                if (Ps==1)|(Ps==0):
                    H = 0
                else:
                    H = -Ps* np.log2(Ps)-Pd*np.log2(Pd)
              #  print('densit:',density)
                if (density <= a): #高密度区域
                    if DRL > 0:
                        Dn.append(list(x))
                else:
                    if (H < H_): #低密度单一标签区域
                        if DRL > 0:
                            Dn.append(list(x))
                           # print('低密度单一标签噪声')
                    else: #低密度混合标签区  
                        DoD_ = density / k
                        if DoD >= DoD_:
                            train_hon,train_hen=[],[]
                            for j in train_set:
                                if j[-1]==x[-1]:
                                    train_hon.append(j)
                                else:
                                    train_hen.append(j)
                            kdt_hon=KdTree()
                            kdt_hen=KdTree()
                            kdt_hon.create(train_hon)
                            kdt_hen.create(train_hen)
                            near_hon=kdt_hon.search(x,k+1)
                            near_hen=kdt_hen.search(x,k)
                            density_hon,density_hen=0,0
                            for hon in near_hon:
                                density_hon += hon[0]
                            for hen in near_hen:
                                density_hen += hen[0]
                            if density_hon / density_hen > 1:
                                Dn.append(list(x))                 
            noise_neighbor=[]                   
            for e in Dn:  # 计算噪声分数
                e=np.array(e)
                near2=kdt.search(e[:-1],k+1) #k近邻
                for i in range(1,k+1):
                    noise_neighbor.append(list(near2[i][1].data))
            for e in Dn:
                e=np.array(e)
                near3=kdt.search(e[:-1],k+1)
                t_e=noise_neighbor.count(list(e)) #处于其它噪声近邻的次数
                confidence_e= 1 / np.sqrt(1+t_e*t_e)#e的置信度
                neighborhood_e=0
                for i in range(1,k+1):
                    t_ei=noise_neighbor.count(list(near3[i][1].data))
                    confidence_ei=1 / np.sqrt(1+t_ei*t_ei) 
                    if near3[i][1].data[-1]==e[-1]:
                        different_class=-1
                    else:
                        different_class=1
                    if list(near3[i][1].data) in Dn: 
                        isnoise=1
                    else:
                        isnoise=-1
                    near4=kdt.search(near3[i][1].data[:-1],k+1)
                    n_ei=0
                    for j in range(1,k+1):
                        if list(near4[j][1].data) in Dn:
                            n_ei += 1
                    clean_ei= (k + isnoise*(n_ei- k)) / (2*k) #纯净度
                    neighborhood_e += (clean_ei*confidence_ei*different_class)/k 
                NS_e=confidence_e * neighborhood_e #噪声得分
                if NS_e > 0:
                    Dn_filted.append(list(e))
            noise_list=noise.values.tolist()
            TP = 0
            for c in Dn_filted:
                if c in noise_list:
                    TP += 1
            FP = len(Dn_filted) - TP
            FN = len(noise_list) - TP
            precision= TP / (TP + FP)
            recall= TP / (TP + FN)
            fh= 1.25* (precision * recall) / (0.25*precision + recall) #F0.5值
            re = len(Dn_filted) / len(train_set)#移除率
            fh_temp.append(fh)
            re_temp.append(re)
        Fh.append(np.mean(fh_temp))
        Re.append(np.mean(re_temp))
    return Fh,Re

#iris数据集

In [4]:
%%time
from sklearn.datasets import load_iris
iris = load_iris()
iris_sample= pd.DataFrame(iris.data, columns=iris.feature_names)
iris_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(iris_sample)))
iris_sample['label'] = iris.target
iris_DDF=DDF(iris_sample)
iris_RDS=RDS(iris_sample)

Wall time: 1min 4s


In [5]:
iris_DDF

([0.6668976789800588,
  0.8234629083627233,
  0.8386866164195552,
  0.8059574780381616,
  0.7581342638295521],
 [0.07259259259259258,
  0.12592592592592594,
  0.24444444444444446,
  0.3718518518518518,
  0.5037037037037038])

In [6]:
iris_RDS

([0.7309049477914736,
  0.8900053077311642,
  0.904610919061609,
  0.863910499316056,
  0.843531542330569],
 [0.06444444444444444,
  0.1162962962962963,
  0.22148148148148147,
  0.30518518518518517,
  0.3874074074074074])

#wine数据集

In [7]:
%%time
from sklearn.datasets import load_wine
wine= load_wine()
wine_sample = pd.DataFrame(wine.data, columns=wine.feature_names)
wine_sample = pd.DataFrame(min_max_scaler.fit_transform(np.array(wine_sample)))
wine_sample['label'] = wine.target
wine_DDF=DDF(wine_sample)
wine_RDS=RDS(wine_sample)

Wall time: 4min 13s


In [8]:
wine_DDF

([0.6142558164818439,
  0.7270105720949049,
  0.7825791973618494,
  0.7644057808860316,
  0.7607841236569132],
 [0.08550077639751552,
  0.14355201863354036,
  0.2627756211180124,
  0.3776552795031056,
  0.5000349378881987])

In [9]:
wine_RDS

([0.7552050936484899,
  0.8539624505928854,
  0.9143110615970468,
  0.8784243715745834,
  0.8436790445195846],
 [0.07116459627329193,
  0.11860636645962734,
  0.21536878881987578,
  0.29276009316770185,
  0.39449534161490685])

#seeds数据集

In [10]:
%%time
seeds=pd.read_csv('C:/Users/liu/python works/mydata/seeds.csv',engine='python')
seeds_sample=seeds.drop(['label'],axis=1)
seeds_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(seeds_sample)))
seeds_sample['label']=seeds.label
seeds_DDF=DDF(seeds_sample)
seeds_RDS=RDS(seeds_sample)

Wall time: 2min 26s


In [11]:
seeds_DDF

([0.5312525205585656,
  0.6542400778265756,
  0.751130297341153,
  0.7235986005599813,
  0.7419203151899387],
 [0.108994708994709,
  0.15978835978835979,
  0.27142857142857146,
  0.3994708994708994,
  0.5021164021164021])

In [12]:
seeds_RDS

([0.5989656647342412,
  0.7390049631295309,
  0.8369258745068956,
  0.8568772071312475,
  0.816129963962841],
 [0.08783068783068784,
  0.13544973544973543,
  0.2312169312169312,
  0.3190476190476191,
  0.4068783068783068])

#glass数据集

In [13]:
%%time
glass=pd.read_csv('C:/Users/liu/python works/mydata/glass.csv')
glass_sample=glass.drop(['label'],axis=1)
glass_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(glass_sample)))
glass_sample['label']=glass.label
glass_DDF=DDF(glass_sample)
glass_RDS=RDS(glass_sample)

Wall time: 3min 22s


In [14]:
glass_DDF

([0.22994196213692236,
  0.3502504601955687,
  0.5152614693294522,
  0.6179656597950369,
  0.617964541090456],
 [0.21841489962105945,
  0.2832580827219221,
  0.36641538337498997,
  0.46163025074578734,
  0.5669676691123116])

In [15]:
glass_RDS

([0.3128657089015378,
  0.4696446013858823,
  0.6143607313213735,
  0.6641479421458776,
  0.6678297000015545],
 [0.16508102878335887,
  0.1999879061517375,
  0.28439087317584455,
  0.37590502297831174,
  0.47871079577521564])

#ecoli数据集

In [16]:
%%time
ecoli=pd.read_csv('C:/Users/liu/python works/mydata/ecoli.csv')
ecoli_sample=ecoli.drop(['label'],axis=1)
ecoli_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(ecoli_sample)))
ecoli_sample['label']=ecoli.label
ecoli_DDF=DDF(ecoli_sample)
ecoli_RDS=RDS(ecoli_sample)

Wall time: 5min 12s


In [17]:
ecoli_DDF

([0.5923190802503946,
  0.7375532605862046,
  0.7668036320900522,
  0.7596573188617407,
  0.6988221849245483],
 [0.09518400802944128,
  0.1376630980260957,
  0.26062562730010036,
  0.3754081632653061,
  0.5257343593174976])

#image-segmentation数据集

In [18]:
%%time
image=pd.read_csv('C:/Users/liu/python works/mydata/segmentation.csv')
image_sample=image.drop(['label'],axis=1)
image_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(image_sample)))
image_sample['label']=image.label
image_DDF=DDF(image_sample)
image_RDS=RDS(image_sample)

Wall time: 9h 27min 41s


In [19]:
image_DDF

([0.5379055956960178,
  0.6992843922310165,
  0.7836599912155218,
  0.7737832378751784,
  0.7557377532146783],
 [0.1012987012987013,
  0.1522847522847523,
  0.2672919672919673,
  0.4057239057239057,
  0.555122655122655])

In [21]:
image_RDS

([0.6375677272487286,
  0.7743025930798952,
  0.8575215207829532,
  0.8678115429922583,
  0.8438951500617764],
 [0.0836940836940837,
  0.13492063492063494,
  0.23915343915343917,
  0.35170755170755175,
  0.48417508417508426])

#statlog数据集

In [22]:
%%time
statlog=pd.read_csv('C:/Users/liu/python works/mydata/statlog.csv',engine='python')
statlog_sample=statlog.drop(['label'],axis=1)
statlog_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(statlog_sample)))
statlog_sample['label']=statlog.label  
statlog_DDF=DDF(statlog_sample)
statlog_RDS=RDS(statlog_sample)

Wall time: 2h 45min 24s


In [23]:
statlog_DDF

([0.16342568610829128,
  0.27576134571863753,
  0.4254894160036963,
  0.5178071721152837,
  0.5830636533047457],
 [0.33792771632849444,
  0.3895423551688102,
  0.48411125021987234,
  0.5797300140373387,
  0.6644346608447924])

#yeast数据集

In [24]:
%%time
yeast=pd.read_csv('C:/Users/liu/python works/mydata/yeast.csv',engine='python')
yeast_sample=yeast.drop(['label'],axis=1)
yeast_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(yeast_sample)))
yeast_sample['label']=yeast.label
yeast_DDF=DDF(yeast_sample)
yeast_RDS=RDS(yeast_sample)

Wall time: 2h 14min 16s


#新甲状腺数据集

In [48]:
%%time
thyroid=pd.read_csv('C:/Users/liu/python works/mydata/new-thyroid.csv',engine='python')
thyroid_sample=thyroid.drop(['label'],axis=1)
thyroid_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(thyroid_sample)))
thyroid_sample['label']=thyroid.label
thyroid_DDF=DDF(thyroid_sample)
thyroid_RDS=RDS(thyroid_sample)

Wall time: 2min 33s


In [50]:
thyroid_DDF

([0.5453787364687114,
  0.6665833358206059,
  0.7785051901383194,
  0.7614884281943747,
  0.7154108321260375],
 [0.10491960899524597,
  0.15503712408525183,
  0.25788152342289405,
  0.37932535655146626,
  0.511620639923081])

In [51]:
thyroid_RDS

([0.563203702285512,
  0.7250103421932208,
  0.8364169620108516,
  0.860105747974513,
  0.832424677029368],
 [0.1018081299075904,
  0.14057475562202876,
  0.22633406335131673,
  0.31316970247315845,
  0.39018481918700926])

#letter数据集

In [27]:
%%time
letter=pd.read_csv('C:/Users/liu/python works/mydata/letter.csv',engine='python')
letter_sample=letter.drop(['label'],axis=1)
letter_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(letter_sample)))
letter_sample['label']=letter.label
letter_DDF=DDF(letter_sample)
letter_RDS=RDS(letter_sample)

Wall time: 5h 6min 13s


#Contraception数据集

In [ ]:
%%time
cmc=pd.read_csv('C:/Users/liu/python works/mydata/Contraception.csv',engine='python')
cmc_sample=cmc.drop(['label'],axis=1)
cmc_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(cmc_sample)))
cmc_sample['label']=cmc.label
cmc_DDF=DDF(cmc_sample)
cmc_RDS=RDS(cmc_sample)

#vowel数据集

In [28]:
%%time
vowel=pd.read_csv('C:/Users/liu/python works/mydata/vowel.csv',engine='python')
vowel_sample=vowel.drop(['label'],axis=1)
vowel_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(vowel_sample)))
vowel_sample['label']=vowel.label
vowel_DDF=DDF(vowel_sample)
vowel_RDS=RDS(vowel_sample)

Wall time: 1h 21min 47s


In [29]:
vowel_DDF

([0.24974480071108013,
  0.3624457298317684,
  0.4707083777258497,
  0.5255983777728146,
  0.5819095633811726],
 [0.2295061728395062,
  0.31,
  0.4648148148148149,
  0.6303703703703704,
  0.7528395061728395])

In [47]:
vowel_RDS

([0.5332004961105561,
  0.6162697747785237,
  0.6709711050620416,
  0.694183098356145,
  0.7025162305143322],
 [0.10160493827160495,
  0.1706172839506173,
  0.30876543209876545,
  0.45518518518518525,
  0.6034567901234568])

#pen_base数据集

In [30]:
%%time
pen_base=pd.read_csv('C:/Users/liu/python works/mydata/pen_base.csv',engine='python')
pen_sample=pen_base.drop(['label'],axis=1)
pen_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(pen_sample)))
pen_sample['label']=pen_base.label
pen_DDF=DDF(pen_sample)
pen_RDS=RDS(pen_sample)

Wall time: 10h 24min 1s


#landsat数据集

In [ ]:
"""
%%time
landsat=pd.read_csv('C:/Users/liu/python works/mydata/landsat.csv',engine='python')
landsat_sample=landsat.drop(['label'],axis=1)
landsat_sample=pd.DataFrame(min_max_scaler.fit_transform(np.array(landsat_sample)))
landsat_sample['label']=landsat.label
landsat_res=DDF(landsat_sample)
"""

In [ ]:
#landsat_res

In [120]:
Fh_DDF=np.array(iris_DDF[0])+np.array(wine_DDF[0])+np.array(seeds_DDF[0])+np.array(glass_DDF[0])+np.array(ecoli_DDF[0])+np.array(image_DDF[0])+np.array(statlog_DDF[0])+np.array(yeast_DDF[0])+np.array(pen_DDF[0])+np.array(letter_DDF[0])

In [121]:
Re_DDF=np.array(iris_DDF[1])+np.array(wine_DDF[1])+np.array(seeds_DDF[1])+np.array(glass_DDF[1])+np.array(ecoli_DDF[1])+np.array(image_DDF[1])+np.array(statlog_DDF[1])+np.array(yeast_DDF[1])+np.array(pen_DDF[1])+np.array(letter_DDF[1])

In [122]:
Fh_RDS=np.array(iris_RDS[0])+np.array(wine_RDS[0])+np.array(seeds_RDS[0])+np.array(glass_RDS[0])+np.array(ecoli_RDS[0])+np.array(image_RDS[0])+np.array(statlog_RDS[0])+np.array(yeast_RDS[0])+np.array(pen_RDS[0])+np.array(letter_RDS[0])

In [123]:
Re_RDS=np.array(iris_RDS[1])+np.array(wine_RDS[1])+np.array(seeds_RDS[1])+np.array(glass_RDS[1])+np.array(ecoli_RDS[1])+np.array(image_RDS[1])+np.array(statlog_RDS[1])+np.array(yeast_RDS[1])+np.array(pen_RDS[1])+np.array(letter_RDS[1])

In [124]:
#Re_DDF=np.array([0.7,1.1,2.0,2.5,3.1])

In [126]:
#Re_RDS=np.array([0.7,1.3,2.2,3.2,4.1])

In [127]:
import mpl_toolkits.axisartist.axislines as axislines
import matplotlib as mpl
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib
noise_rate=[5,10,20,30,40]
fig = plt.figure(figsize=(5,10))
ax1 = axislines.Subplot(fig, 2,1,1)
fig.add_subplot(ax1)
ax1.set_xticks(noise_rate)
ax1.axis([0,40,0,1])
plt.plot(noise_rate,Fh_DDF/10,'r.-',label='DDF')
plt.plot(noise_rate,Fh_RDS/10,'b*-',label='DDF-NOS')
plt.xlabel('noise rate/% ')
plt.ylabel('F0.5')
#plt.title('不同噪声下算法的过滤性能')
plt.legend()
plt.show()
ax2 = axislines.Subplot(fig, 2,1,2)
fig.add_subplot(ax2)
ax2.set_xticks([5,10,20,30,40])
ax2.set_yticks([0,0.2,0.4,0.6,0.8])
ax2.axis([0,40,0,0.8])
plt.plot(noise_rate,Re_DDF/10,'r.-',label='DDF')
plt.plot(noise_rate,Re_RDS/10,'b*-',label='DDF-NOS')
plt.xlabel('noise rate/%')
plt.ylabel('Re')
# plt.title('不同噪声下的过滤值Re',fontsize=12)
plt.legend()
plt.show()
plt.tight_layout(2) #设置子图间隔
#plt.suptitle(title)
plt.show()
plt.savefig(r'./对比实验结果图F0.5/10个数据集.png')

Using matplotlib backend: Qt5Agg
